In [122]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.base import BaseEstimator,TransformerMixin
warnings.filterwarnings('ignore')
df=pd.read_csv(r"C:\Users\HP\Downloads\laptop_data.csv").drop(['Unnamed: 0'],axis=1)
df

,Company,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price
0,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,71378.6832
1,Apple,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,47895.5232
2,HP,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,30636.0000
3,Apple,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,135195.3360
4,Apple,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,96095.8080
...,...,...,...,...,...,...,...,...,...,...,...
1298,Lenovo,2 in 1 Convertible,14.0,IPS Panel Full HD / Touchscreen 1920x1080,Intel Core i7 6500U 2.5GHz,4GB,128GB SSD,Intel HD Graphics 520,Windows 10,1.8kg,33992.6400
1299,Lenovo,2 in 1 Convertible,13.3,IPS Panel Quad HD+ / Touchscreen 3200x1800,Intel Core i7 6500U 2.5GHz,16GB,512GB SSD,Intel HD Graphics 520,Windows 10,1.3kg,79866.7200
1300,Lenovo,Notebook,14.0,1366x768,Intel Celeron Dual Core N3050 1.6GHz,2GB,64GB Flash Storage,Intel HD Graphics,Windows 10,1.5kg,12201.1200
1301,HP,Notebook,15.6,1366x768,Intel Core i7 6500U 2.5GHz,6GB,1TB HDD,AMD Radeon R5 M330,Windows 10,2.19kg,40705.9200


In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Company           1303 non-null   object 
 1   TypeName          1303 non-null   object 
 2   Inches            1303 non-null   float64
 3   ScreenResolution  1303 non-null   object 
 4   Cpu               1303 non-null   object 
 5   Ram               1303 non-null   object 
 6   Memory            1303 non-null   object 
 7   Gpu               1303 non-null   object 
 8   OpSys             1303 non-null   object 
 9   Weight            1303 non-null   object 
 10  Price             1303 non-null   float64
dtypes: float64(2), object(9)
memory usage: 112.1+ KB


In [124]:
class Preprocessing(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    
    def processor(self,text):
        self.text = text
        try:
            if "Intel Core" in self.text:
                return self.text
            elif "Intel" not in self.text:
                return "Non Intel Processor"
            else:
                return "Other Intel Processor"
        except:
            return np.nan
    
    def SSD_func(self,text):
        self.text = text
        try:
            if type(self.text)==str:
                if 'GB' in self.text:
                    return int(self.text.split('GB')[0])
                elif 'TB' in self.text:
                    return int(self.text.split('TB')[0])*1000
                else:
                    return self.text
            else:
                return self.text
        except:
            return np.nan
        
    
    def HDD_func(self,text):
        self.text=text
        try:
            if type(self.text)==int:
                return self.text
            else:
                if 'TB' in self.text:
                    return int(self.text.replace('TB', ''))*1000
                elif 'GB' in self.text:
                    return int(self.text.replace('GB', ''))
                else:
                    return 0
        except:
            return 0
        
    def OS(self,text):
        self.text=text
        try:
            if type(self.text) == str:
                if 'Windows 10' in text or 'Windows 7' in text or 'Windows 10 S' in self.text:
                    return "Windos"
                elif 'macOS'in text or 'Mac OS X' in self.text:
                    return "Mac"
                else:
                    return "No OS/Chrome OS/Android"

        except:
            return np.nan
        
    def GPU(self,text):
        self.text = text
        try:
            if type(text) == str:
                if 'Intel' in self.text:
                    return 'Intel'
                elif 'Nvidia' in self.text:
                    return 'Nvidia'
                elif 'AMD' in self.text:
                    return 'AMD'
                else:
                    return 'Other' 
        except:
            return np.nan
    
    
    def transform(self,df):
        df['Ram']=df['Ram'].str.replace('GB','').astype(int)
        df['Weight']=df['Weight'].str.replace('kg','').astype(float)
        df['TouchScreen']=df['ScreenResolution'].str.count('Touchscreen')
        df['x-res']=df['ScreenResolution'].str.split('x').str.get(0).str.split(" ").str.get(-1).astype(int)
        df['y-res']=df['ScreenResolution'].str.split('x').str.get(1).str.strip().astype(int)
        df['ppi']=round((((df['x-res']**2)+(df['y-res']**2))**.5)/df['Inches'],2)
        df['Price']=round(df['Price'],0)
        df['Cpu Name']=df['Cpu'].str.split(' ').apply(lambda x:" ".join(x[:3]))
        df['Cpu Name']=df['Cpu Name'].apply(self.processor)
        df['SDD']=df['Memory'].apply(lambda x:x.split("SSD")[0] if "SSD" in x else 0)
        df.drop(['ScreenResolution','Cpu','x-res','y-res'],axis=1,inplace=True)
        df['SDD']=df['SDD'].apply(self.SSD_func)
        df['HDD']=df['Memory'].apply(lambda x:x.split("HDD") if "HDD" in x else 0)
        df['HDD']=df['HDD'].apply(lambda x:x[0].split('+')[-1].strip() if type(x)==list else x)
        df['HDD']=df['HDD'].apply(self.HDD_func)
        df["Flash Storage"]=df['Memory'].apply(lambda x: int(x.split(" ")[0].split("G")[0]) if "Flash Storage" in x else 0)
        df.drop(["Memory"],axis=1,inplace=True)
        df['OS']=df['OpSys'].apply(self.OS)
        df.drop(['OpSys'],axis=1,inplace=True)
        df['Gpu']=df['Gpu'].apply(self.GPU)
        return df
    
    

In [125]:
trans=Preprocessing()
df=trans.transform(df)
df

,Company,TypeName,Inches,Ram,Gpu,Weight,Price,TouchScreen,ppi,Cpu Name,SDD,HDD,Flash Storage,OS
0,Apple,Ultrabook,13.3,8,Intel,1.37,71379.0,0,226.98,Intel Core i5,128,0,0,Mac
1,Apple,Ultrabook,13.3,8,Intel,1.34,47896.0,0,127.68,Intel Core i5,0,0,128,Mac
2,HP,Notebook,15.6,8,Intel,1.86,30636.0,0,141.21,Intel Core i5,256,0,0,No OS/Chrome OS/Android
3,Apple,Ultrabook,15.4,16,AMD,1.83,135195.0,0,220.53,Intel Core i7,512,0,0,Mac
4,Apple,Ultrabook,13.3,8,Intel,1.37,96096.0,0,226.98,Intel Core i5,256,0,0,Mac
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298,Lenovo,2 in 1 Convertible,14.0,4,Intel,1.80,33993.0,1,157.35,Intel Core i7,128,0,0,Windos
1299,Lenovo,2 in 1 Convertible,13.3,16,Intel,1.30,79867.0,1,276.05,Intel Core i7,512,0,0,Windos
1300,Lenovo,Notebook,14.0,2,Intel,1.50,12201.0,0,111.94,Other Intel Processor,0,0,64,Windos
1301,HP,Notebook,15.6,6,AMD,2.19,40706.0,0,100.45,Intel Core i7,0,1000,0,Windos


In [126]:
cat_col=['Company','TypeName','Gpu','Cpu Name','OS']
num_col=['Inches','Ram','Weight','TouchScreen','ppi','SDD','HDD','Flash Storage']

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor

In [127]:
ct=ColumnTransformer(transformers=[('CT',OneHotEncoder(sparse_output=False,drop='first'),cat_col),
                                   ('SD',StandardScaler(with_mean=True, with_std=True),num_col)]
                     ,remainder='passthrough')

In [128]:
y=df.iloc[:,6]
X=df.drop(['Price'],axis=1)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=1388)

In [129]:
from sklearn.pipeline import Pipeline
GBR=GradientBoostingRegressor(random_state=1388,n_estimators=100,max_depth=3,min_samples_leaf=1,min_samples_split=2,alpha=.9)
pipe=Pipeline([('Transform',ct),('GBR',GBR)])

<IPython.core.display.Javascript object>

In [130]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('Transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('CT',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['Company', 'TypeName', 'Gpu',
                                                   'Cpu Name', 'OS']),
                                                 ('SD', StandardScaler(),
                                                  ['Inches', 'Ram', 'Weight',
                                                   'TouchScreen', 'ppi', 'SDD',
                                                   'HDD', 'Flash Storage'])])),
                ('GBR', GradientBoostingRegressor(random_state=1388))])

In [131]:
y_pred=pipe.predict(X_test)

In [132]:
from sklearn.metrics import r2_score

In [133]:
r2_score(y_test,y_pred)

0.8203204231814522

In [134]:
from sklearn.metrics import mean_absolute_error

In [135]:
mean_absolute_error(y_test,y_pred)

10629.39392384596

In [136]:
from tensorflow import keras

y=df.iloc[:,6]
X=df.drop(['Price'],axis=1)
X=ct.fit_transform(X)
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=1388)

In [137]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
model = Sequential()

In [138]:
model.add(Dense(100,input_dim=41,activation='relu'))
model.add(Dropout(.2))
model.add(Dense(50,activation='relu'))
model.add(Dropout(.25))
model.add(Dense(50,activation='relu'))
model.add(Dropout(0.20))
model.add(Dense(1,activation='linear'))

In [139]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 100)               4200      
                                                                 
 dropout_18 (Dropout)        (None, 100)               0         
                                                                 
 dense_25 (Dense)            (None, 50)                5050      
                                                                 
 dropout_19 (Dropout)        (None, 50)                0         
                                                                 
 dense_26 (Dense)            (None, 50)                2550      
                                                                 
 dropout_20 (Dropout)        (None, 50)                0         
                                                                 
 dense_27 (Dense)            (None, 1)                

In [140]:
model.compile(loss='mse', optimizer='adam', metrics=['mean_squared_error'])

In [141]:
model.fit(X_train, y_train,epochs=75,validation_split=.3)

Epoch 1/75
23/23 [==============================] - 5s 21ms/step - loss: 5231187968.0000 - mean_squared_error: 5231187968.0000 - val_loss: 4589291008.0000 - val_mean_squared_error: 4589291008.0000
Epoch 2/75
23/23 [==============================] - 0s 7ms/step - loss: 5230772736.0000 - mean_squared_error: 5230772736.0000 - val_loss: 4588549120.0000 - val_mean_squared_error: 4588549120.0000
Epoch 3/75
23/23 [==============================] - 0s 7ms/step - loss: 5228827648.0000 - mean_squared_error: 5228827648.0000 - val_loss: 4585035264.0000 - val_mean_squared_error: 4585035264.0000
Epoch 4/75
23/23 [==============================] - 0s 7ms/step - loss: 5220686848.0000 - mean_squared_error: 5220686848.0000 - val_loss: 4572036096.0000 - val_mean_squared_error: 4572036096.0000
Epoch 5/75
23/23 [==============================] - 0s 7ms/step - loss: 5194625536.0000 - mean_squared_error: 5194625536.0000 - val_loss: 4533843968.0000 - val_mean_squared_error: 4533843968.0000
Epoch 6/75
23/23 [=

In [142]:
y_pred_ann=model.predict(X_test)

9/9 [==============================] - 0s 4ms/step


In [143]:
y_pred_ann

array([[100169.61  ],
       [ 61414.86  ],
       [ 64774.355 ],
       [110526.69  ],
       [ 59238.723 ],
       [ 67571.17  ],
       [ 97422.94  ],
       [ 33046.953 ],
       [ 23177.527 ],
       [ 48833.062 ],
       [109884.375 ],
       [ 37210.6   ],
       [ 49451.77  ],
       [ 66138.07  ],
       [ 62442.11  ],
       [ 54809.426 ],
       [ 57074.684 ],
       [ 67956.125 ],
       [ 40353.332 ],
       [ 53174.95  ],
       [ 49891.625 ],
       [ 92926.01  ],
       [ 53786.363 ],
       [ 60581.535 ],
       [ 95957.18  ],
       [ 30165.496 ],
       [ 16380.156 ],
       [ 94800.87  ],
       [ 51321.445 ],
       [ 54621.113 ],
       [140121.81  ],
       [ 35113.18  ],
       [110148.4   ],
       [ 22193.705 ],
       [ 60460.14  ],
       [ 59928.305 ],
       [ 40740.1   ],
       [ 89820.83  ],
       [ 47157.31  ],
       [ 11266.46  ],
       [ 16311.165 ],
       [ 42038.684 ],
       [ 21685.191 ],
       [ 29911.742 ],
       [ 72727.48  ],
       [ 3

In [144]:
r2_score(y_test,y_pred_ann)

0.754764504946218